In [1]:
import augur
from augur.PiperReader import PiperMail

# import everything from githubapi.py and ghtorrent.py so we can
# just copy and paste our function later
import json
import pandas as pd
from perceval.backends.core.pipermail import Pipermail
import os, os.path

# create an Augur application so we can test our function
print(os.getcwd())
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
print(os.getcwd())
link = "https://lists.opendaylight.org/pipermail/"
mail = ["aalldp-dev","aaa-dev","advisory-group","affinity-dev","alto-dev","archetypes-dev"]
#print(os.getcwd())
file = "opendaylight-"
if "notebooks" in os.getcwd():
    os.chdir("..")
path = "/augur/data/opendaylight-" 
for x in range(len(mail)):
    #print(link+mail[x])
    #print(os.getcwd())
    #print(os.path.join(os.getcwd() + path+'.json'))
    place = os.path.join(os.getcwd() + path + mail[x] +'.json')
    if(os.path.exists(place)):
        print(mail[x])
        print("File Exists")
        continue                   
    repo = Pipermail(url = "https://lists.opendaylight.org/pipermail/"+ mail[x] + "/",dirpath="tmp/archives"+mail[x])
    #print(repo)
    outfile = open(place,"w+")
    for message in repo.fetch():
        obj = json.dumps(message, indent=4, sort_keys=True)
        outfile.write(obj)
        outfile.write('\n')
    outfile.close()
    print("Created File",mail[x])
print("Finished downloading files")
print(link)

/home/keanu/temp/augur7/augur/notebooks
/home/keanu/temp/augur7/augur/notebooks
aalldp-dev
File Exists
aaa-dev
File Exists
advisory-group
File Exists
affinity-dev
File Exists
alto-dev
File Exists
archetypes-dev
File Exists
Finished downloading files
https://lists.opendaylight.org/pipermail/


In [2]:
print(link)
temp = augurApp.piper()
temp.make(link)
print(temp)
#temp.make()
#print(temp.make)

https://lists.opendaylight.org/pipermail/
ugh
https://lists.opendaylight.org/pipermail/
Hey
File uploaded  2
File uploaded  855
File uploaded  546
True
Finished
